In [1]:
import os

In [2]:
%pwd

'd:\\text-summarizer-project\\research'

In [3]:
os.chdir("../")

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/kiranaradhya20/text-summarizer-project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="kiranaradhya20"
os.environ["MLFLOW_TRACKING_PASSWORD"]="Geetha2000$"

In [6]:
'''MLFLOW_TRACKING_URI=https://dagshub.com/kiranaradhya20/text-summarizer-project.mlflow \
MLFLOW_TRACKING_USERNAME=kiranaradhya20 \
MLFLOW_TRACKING_PASSWORD=15353351c9c000dec7bdda258e97a8d254a9233d \
python script.py'''

'MLFLOW_TRACKING_URI=https://dagshub.com/kiranaradhya20/text-summarizer-project.mlflow MLFLOW_TRACKING_USERNAME=kiranaradhya20 MLFLOW_TRACKING_PASSWORD=15353351c9c000dec7bdda258e97a8d254a9233d python script.py'

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path
    all_params: dict
    mlflow_uri: str

In [8]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories


In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path = config.model_path,
            tokenizer_path = config.tokenizer_path,
            metric_file_name = config.metric_file_name,
            mlflow_uri="https://dagshub.com/kiranaradhya20/text-summarizer-project.mlflow",
            all_params=self.params
           
        )

        return model_evaluation_config


In [11]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk, load_metric
import torch
import pandas as pd
from tqdm import tqdm
import mlflow
import mlflow.keras
from urllib.parse import urlparse

c:\Users\Kiran Aradhya\anaconda3\envs\textS1\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-01-03 20:40:17,620: INFO: config: PyTorch version 2.1.2 available.]
[2024-01-03 20:40:17,624: INFO: config: TensorFlow version 2.13.0 available.]


In [14]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config


    
    def generate_batch_sized_chunks(self,list_of_elements, batch_size):
        """split the dataset into smaller batches that we can process simultaneously
        Yield successive batch-sized chunks from list_of_elements."""
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]

    
    def calculate_metric_on_test_ds(self,dataset, metric, model, tokenizer, 
                               batch_size=16, device="cuda" if torch.cuda.is_available() else "cpu", 
                               column_text="article", 
                               column_summary="highlights"):
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)):
            
            inputs = tokenizer(article_batch, max_length=1024,  truncation=True, 
                            padding="max_length", return_tensors="pt")
            
            summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                            attention_mask=inputs["attention_mask"].to(device), 
                            length_penalty=0.8, num_beams=8, max_length=128)
            ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''
            
            # Finally, we decode the generated texts, 
            # replace the  token, and add the decoded texts with the references to the metric.
            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, 
                                    clean_up_tokenization_spaces=True) 
                for s in summaries]      
            
            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
            
            
            metric.add_batch(predictions=decoded_summaries, references=target_batch)
            
        #  Finally compute and return the ROUGE scores.
        score = metric.compute()
        
        return score


    def evaluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)
       
        #loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)


        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
  
        rouge_metric = load_metric('rouge')

        score = self.calculate_metric_on_test_ds(
        dataset_samsum_pt['test'][0:10], rouge_metric, model_pegasus, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
            )

        rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )
        lis=[]
        rouge_list = [lis.append(score[rn].mid.fmeasure) for rn in rouge_names ]
        print(lis[0],lis[1],lis[2],lis[3])
        

        df = pd.DataFrame(rouge_dict, index = ['pegasus'] )
        df.to_csv(self.config.metric_file_name, index=False)
        return lis,model_pegasus

    def log_into_mlflow(self,lis,model):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"rouge1": lis[0], "rouge2": lis[1],"rougeL": lis[2],"rougeLsum": lis[3]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.pytorch.log_model(model, "model", registered_model_name="pegasus")
            else:
                mlflow.pytorch.log_model(model, "model")



In [15]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    lis,model = model_evaluation_config.evaluate()
    model_evaluation_config.log_into_mlflow(lis,model)
except Exception as e:
    raise e

[2024-01-03 20:46:40,403: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-03 20:46:40,405: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-03 20:46:40,407: INFO: common: created directory at: artifacts]
[2024-01-03 20:46:40,409: INFO: common: created directory at: artifacts/model_evaluation]


c:\Users\Kiran Aradhya\anaconda3\envs\textS1\lib\site-packages\datasets\load.py:752: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
Using the latest cached version of the module from C:\Users\Kiran Aradhya\.cache\huggingface\modules\datasets_modules\metrics\rouge\08e5f021b5761265deaafbf424e57913106427f546189fe3f934069dd32c153f (last modified on Sat Dec 30 23:03:28 2023) since it couldn't be found locally at rouge, or remotely on the Hugging Face Hub.


[2024-01-03 20:49:38,532: WARNING: load: Using the latest cached version of the module from C:\Users\Kiran Aradhya\.cache\huggingface\modules\datasets_modules\metrics\rouge\08e5f021b5761265deaafbf424e57913106427f546189fe3f934069dd32c153f (last modified on Sat Dec 30 23:03:28 2023) since it couldn't be found locally at rouge, or remotely on the Hugging Face Hub.]


100%|██████████| 5/5 [02:28<00:00, 29.64s/it]

[2024-01-03 20:52:06,744: INFO: rouge_scorer: Using default tokenizer.]


0.02002092483673442 0.0 0.019903641968459104 0.01966811637297065


c:\Users\Kiran Aradhya\anaconda3\envs\textS1\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'pegasus' already exists. Creating a new version of this model...
2024/01/03 21:15:38 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: pegasus, version 2
Created version '2' of model 'pegasus'.
